In [ ]:
import optuna
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import accuracy_score, matthews_corrcoef, confusion_matrix, roc_auc_score, precision_recall_curve, auc
import numpy as np
from numpy import mean
from collections import Counter
import re, os, sys
import random
import pandas as pd
import lightgbm as lgb
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, confusion_matrix, roc_curve, roc_auc_score, f1_score, matthews_corrcoef
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
import shap
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import svm
import lightgbm as lgb
import optuna
from sklearn import tree
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from numpy import mean
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import random
from matplotlib import pyplot
from numpy import sort
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
import lightgbm as lgb
from sklearn.metrics import confusion_matrix, recall_score, roc_curve, roc_auc_score, auc
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectKBest, chi2, f_classif

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import permutation_importance
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from catboost import CatBoostClassifier
import iFeatureOmegaCLI


In [ ]:
def extract_feature(input_txt_path, feature_name):
    feature = iFeatureOmegaCLI.iRNA(input_txt_path)

    feature.get_descriptor(feature_name)

    return feature.encodings.reset_index(drop=True)

feature_names = [
    "CKSNAP type 1", "Z_curve_48bit", "Kmer type 1", "Mismatch", "DBE", 
    "ENAC", "NAC", "MMI", "NCP", "PS2", "ASDC"
]

inputfile = '/train_drosophila.fasta'

all_features = {}

for feature_name in feature_names:
    feature_data = extract_feature(inputfile, feature_name)
    all_features[feature_name] = feature_data

    length = len(feature_data)
    pos_lab = np.ones(int(length / 2))
    neg_lab = np.zeros(int(length / 2))
    new_column = np.concatenate((pos_lab, neg_lab), axis=0)
    
    feature_data['new_column'] = new_column
    labels = feature_data['new_column'].to_numpy()

    data_io = feature_data.drop(columns=['new_column']).to_numpy()

    '''
    c = list(zip(data_only, labels))
    random.Random(100).shuffle(c)
    data_io, labels = zip(*c)
    '''

    feature_key = feature_name.lower().replace(" type 1", "").replace("_48bit", "").replace(" ", "_").replace("2","")
    globals()[f"data_{feature_key}"] = np.asarray(data_io)
    globals()[f"labels_{feature_key}"] = np.asarray(labels)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.cross_decomposition import PLSRegression

xgb_cksnap = XGBClassifier(learning_rate=0.017965145536083, max_depth=11, 
                min_child_weight=1, gamma=0.17426232112256684, 
                colsample_bytree=0.1916286359142644, n_estimators=625, seed=14)

lgbm_cksnap = lgb.LGBMClassifier(learning_rate=0.011038126658226765, max_depth=12,
                min_child_samples=5, min_child_weight=0.4318851550452938, min_split_gain=0.025127530387291663,
                n_estimators=471, num_leaves=31) 

svm_cksnap = SVC(C=2, kernel='rbf', gamma='scale')

rf_cksnap = RandomForestClassifier(max_depth=80, bootstrap=False, max_features = 'sqrt',
                                    n_estimators=400, min_samples_split=3,random_state=0)
lg_cksnap = LogisticRegression(penalty='l2', C=1.0, max_iter=1000, solver='lbfgs', random_state=14)

mlp_cksnap = MLPClassifier(hidden_layer_sizes=(210,), activation='logistic', solver='adam', random_state=10)

nb_cksnap = GaussianNB()  

knn_cksnap = KNeighborsClassifier(n_neighbors=7, weights='distance', p=2)  

ada_cksnap = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=1),
    n_estimators=100, learning_rate=0.5, random_state=0
)  

dt_cksnap = DecisionTreeClassifier(
    max_depth=20, min_samples_split=4, min_samples_leaf=2, random_state=0
)  

et_cksnap = ExtraTreesClassifier(
    n_estimators=200, max_depth=15, min_samples_split=4, min_samples_leaf=2,
    max_features='sqrt', bootstrap=True, random_state=0
)  
cb_cksnap = CatBoostClassifier(learning_rate=0.25659939142290733, max_depth=4,iterations=90,
                                  boosting_type='Plain')
    
classifiers_cksnap = [lgbm_cksnap, svm_cksnap, lg_cksnap, mlp_cksnap, knn_cksnap, ada_cksnap, et_cksnap, cb_cksnap, nb_cksnap, xgb_cksnap, rf_cksnap]

######

xgb_Z_curve = XGBClassifier(learning_rate=0.017965145536083, max_depth=11, 
                min_child_weight=1, gamma=0.17426232112256684, 
                colsample_bytree=0.1916286359142644, n_estimators=625, seed=14)

lgbm_Z_curve = lgb.LGBMClassifier(learning_rate=0.011038126658226765, max_depth=12,
                min_child_samples=5, min_child_weight=0.4318851550452938, min_split_gain=0.025127530387291663,
                n_estimators=471, num_leaves=31) 

svm_Z_curve = SVC(C=2, kernel='rbf', gamma='scale')

rf_Z_curve = RandomForestClassifier(max_depth=80, bootstrap=False, max_features = 'sqrt',
                                    n_estimators=400, min_samples_split=3,random_state=0)
lg_Z_curve = LogisticRegression(penalty='l2', C=1.0, max_iter=1000, solver='lbfgs', random_state=14)

mlp_Z_curve = MLPClassifier(hidden_layer_sizes=(210,), activation='logistic', solver='adam', random_state=10)

nb_Z_curve = GaussianNB()  

knn_Z_curve = KNeighborsClassifier(n_neighbors=7, weights='distance', p=2)  

ada_Z_curve = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=1),
    n_estimators=100, learning_rate=0.5, random_state=0
)  

dt_Z_curve = DecisionTreeClassifier(
    max_depth=20, min_samples_split=4, min_samples_leaf=2, random_state=0
)  

et_Z_curve = ExtraTreesClassifier(
    n_estimators=200, max_depth=15, min_samples_split=4, min_samples_leaf=2,
    max_features='sqrt', bootstrap=True, random_state=0
)  
cb_Z_curve = CatBoostClassifier(learning_rate=0.25659939142290733, max_depth=4,iterations=90,
                                  boosting_type='Plain')
    
classifiers_Z_curve = [lgbm_Z_curve, svm_Z_curve, lg_Z_curve, mlp_Z_curve, knn_Z_curve, ada_Z_curve, et_Z_curve, cb_Z_curve, nb_Z_curve, xgb_Z_curve, rf_Z_curve]

###
xgb_kmer = XGBClassifier(learning_rate=0.017965145536083, max_depth=11, 
                min_child_weight=1, gamma=0.17426232112256684, 
                colsample_bytree=0.1916286359142644, n_estimators=625, seed=14)

lgbm_kmer = lgb.LGBMClassifier(learning_rate=0.011038126658226765, max_depth=12,
                min_child_samples=5, min_child_weight=0.4318851550452938, min_split_gain=0.025127530387291663,
                n_estimators=471, num_leaves=31) 

svm_kmer = SVC(C=2, kernel='rbf', gamma='scale')

rf_kmer = RandomForestClassifier(max_depth=80, bootstrap=False, max_features = 'sqrt',
                                    n_estimators=400, min_samples_split=3,random_state=0)
lg_kmer = LogisticRegression(penalty='l2', C=1.0, max_iter=1000, solver='lbfgs', random_state=14)

mlp_kmer = MLPClassifier(hidden_layer_sizes=(210,), activation='logistic', solver='adam', random_state=10)

nb_kmer = GaussianNB()  

knn_kmer = KNeighborsClassifier(n_neighbors=7, weights='distance', p=2)  

ada_kmer = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=1),
    n_estimators=100, learning_rate=0.5, random_state=0
)  

dt_kmer = DecisionTreeClassifier(
    max_depth=20, min_samples_split=4, min_samples_leaf=2, random_state=0
)  

et_kmer = ExtraTreesClassifier(
    n_estimators=200, max_depth=15, min_samples_split=4, min_samples_leaf=2,
    max_features='sqrt', bootstrap=True, random_state=0
)  
cb_kmer = CatBoostClassifier(learning_rate=0.25659939142290733, max_depth=4,iterations=90,
                                  boosting_type='Plain')
    
classifiers_kmer = [lgbm_kmer, svm_kmer, lg_kmer, mlp_kmer, knn_kmer, ada_kmer, et_kmer, cb_kmer, nb_kmer, xgb_kmer, rf_kmer]

####

xgb_mismatch = XGBClassifier(learning_rate=0.017965145536083, max_depth=11, 
                min_child_weight=1, gamma=0.17426232112256684, 
                colsample_bytree=0.1916286359142644, n_estimators=625, seed=14)

lgbm_mismatch = lgb.LGBMClassifier(learning_rate=0.011038126658226765, max_depth=12,
                min_child_samples=5, min_child_weight=0.4318851550452938, min_split_gain=0.025127530387291663,
                n_estimators=471, num_leaves=31) 

svm_mismatch = SVC(C=2, kernel='rbf', gamma='scale')

rf_mismatch = RandomForestClassifier(max_depth=80, bootstrap=False, max_features = 'sqrt',
                                    n_estimators=400, min_samples_split=3,random_state=0)
lg_mismatch = LogisticRegression(penalty='l2', C=1.0, max_iter=1000, solver='lbfgs', random_state=14)

mlp_mismatch = MLPClassifier(hidden_layer_sizes=(210,), activation='logistic', solver='adam', random_state=10)

nb_mismatch = GaussianNB()  

knn_mismatch = KNeighborsClassifier(n_neighbors=7, weights='distance', p=2)  

ada_mismatch = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=1),
    n_estimators=100, learning_rate=0.5, random_state=0
)  

dt_mismatch = DecisionTreeClassifier(
    max_depth=20, min_samples_split=4, min_samples_leaf=2, random_state=0
)  

et_mismatch = ExtraTreesClassifier(
    n_estimators=200, max_depth=15, min_samples_split=4, min_samples_leaf=2,
    max_features='sqrt', bootstrap=True, random_state=0
)  
cb_mismatch = CatBoostClassifier(learning_rate=0.25659939142290733, max_depth=4,iterations=90,
                                  boosting_type='Plain')
    
classifiers_mismatch = [lgbm_mismatch, svm_mismatch, lg_mismatch, mlp_mismatch, knn_mismatch, ada_mismatch, et_mismatch, cb_mismatch, nb_mismatch, xgb_mismatch, rf_mismatch]

###
xgb_dbe = XGBClassifier(learning_rate=0.017965145536083, max_depth=11, 
                min_child_weight=1, gamma=0.17426232112256684, 
                colsample_bytree=0.1916286359142644, n_estimators=625, seed=14)

lgbm_dbe = lgb.LGBMClassifier(learning_rate=0.011038126658226765, max_depth=12,
                min_child_samples=5, min_child_weight=0.4318851550452938, min_split_gain=0.025127530387291663,
                n_estimators=471, num_leaves=31) 

svm_dbe = SVC(C=2, kernel='rbf', gamma='scale')

rf_dbe = RandomForestClassifier(max_depth=80, bootstrap=False, max_features = 'sqrt',
                                    n_estimators=400, min_samples_split=3,random_state=0)
lg_dbe = LogisticRegression(penalty='l2', C=1.0, max_iter=1000, solver='lbfgs', random_state=14)

mlp_dbe = MLPClassifier(hidden_layer_sizes=(210,), activation='logistic', solver='adam', random_state=10)

nb_dbe = GaussianNB()  

knn_dbe = KNeighborsClassifier(n_neighbors=7, weights='distance', p=2)  

ada_dbe = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=1),
    n_estimators=100, learning_rate=0.5, random_state=0
)  

dt_dbe = DecisionTreeClassifier(
    max_depth=20, min_samples_split=4, min_samples_leaf=2, random_state=0
)  

et_dbe = ExtraTreesClassifier(
    n_estimators=200, max_depth=15, min_samples_split=4, min_samples_leaf=2,
    max_features='sqrt', bootstrap=True, random_state=0
)  
cb_dbe = CatBoostClassifier(learning_rate=0.25659939142290733, max_depth=4,iterations=90,
                                  boosting_type='Plain')
    
classifiers_dbe = [lgbm_dbe, svm_dbe, lg_dbe, mlp_dbe, knn_dbe, ada_dbe, et_dbe, cb_dbe, nb_dbe, xgb_dbe, rf_dbe]

###
xgb_enac = XGBClassifier(learning_rate=0.017965145536083, max_depth=11, 
                min_child_weight=1, gamma=0.17426232112256684, 
                colsample_bytree=0.1916286359142644, n_estimators=625, seed=14)

lgbm_enac = lgb.LGBMClassifier(learning_rate=0.011038126658226765, max_depth=12,
                min_child_samples=5, min_child_weight=0.4318851550452938, min_split_gain=0.025127530387291663,
                n_estimators=471, num_leaves=31) 

svm_enac = SVC(C=2, kernel='rbf', gamma='scale')

rf_enac = RandomForestClassifier(max_depth=80, bootstrap=False, max_features = 'sqrt',
                                    n_estimators=400, min_samples_split=3,random_state=0)
lg_enac = LogisticRegression(penalty='l2', C=1.0, max_iter=1000, solver='lbfgs', random_state=14)

mlp_enac = MLPClassifier(hidden_layer_sizes=(210,), activation='logistic', solver='adam', random_state=10)

nb_enac = GaussianNB()  

knn_enac = KNeighborsClassifier(n_neighbors=7, weights='distance', p=2)  

ada_enac = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=1),
    n_estimators=100, learning_rate=0.5, random_state=0
)  

dt_enac = DecisionTreeClassifier(
    max_depth=20, min_samples_split=4, min_samples_leaf=2, random_state=0
)  

et_enac = ExtraTreesClassifier(
    n_estimators=200, max_depth=15, min_samples_split=4, min_samples_leaf=2,
    max_features='sqrt', bootstrap=True, random_state=0
)  
cb_enac = CatBoostClassifier(learning_rate=0.25659939142290733, max_depth=4,iterations=90,
                                  boosting_type='Plain')
    
classifiers_enac = [lgbm_enac, svm_enac, lg_enac, mlp_enac, knn_enac, ada_enac, et_enac, cb_enac, nb_enac, xgb_enac, rf_enac]

####

xgb_nac = XGBClassifier(learning_rate=0.017965145536083, max_depth=11, 
                min_child_weight=1, gamma=0.17426232112256684, 
                colsample_bytree=0.1916286359142644, n_estimators=625, seed=14)

lgbm_nac = lgb.LGBMClassifier(learning_rate=0.011038126658226765, max_depth=12,
                min_child_samples=5, min_child_weight=0.4318851550452938, min_split_gain=0.025127530387291663,
                n_estimators=471, num_leaves=31) 

svm_nac = SVC(C=2, kernel='rbf', gamma='scale')

rf_nac = RandomForestClassifier(max_depth=80, bootstrap=False, max_features = 'sqrt',
                                    n_estimators=400, min_samples_split=3,random_state=0)
lg_nac = LogisticRegression(penalty='l2', C=1.0, max_iter=1000, solver='lbfgs', random_state=14)

mlp_nac = MLPClassifier(hidden_layer_sizes=(210,), activation='logistic', solver='adam', random_state=10)

nb_nac = GaussianNB()  

knn_nac = KNeighborsClassifier(n_neighbors=7, weights='distance', p=2)  

ada_nac = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=1),
    n_estimators=100, learning_rate=0.5, random_state=0
)  

dt_nac = DecisionTreeClassifier(
    max_depth=20, min_samples_split=4, min_samples_leaf=2, random_state=0
)  

et_nac = ExtraTreesClassifier(
    n_estimators=200, max_depth=15, min_samples_split=4, min_samples_leaf=2,
    max_features='sqrt', bootstrap=True, random_state=0
)  
cb_nac = CatBoostClassifier(learning_rate=0.25659939142290733, max_depth=4,iterations=90,
                                  boosting_type='Plain')
    
classifiers_nac = [lgbm_nac, svm_nac, lg_nac, mlp_nac, knn_nac, ada_nac, et_nac, cb_nac, nb_nac, xgb_nac, rf_nac]

###
xgb_mmi = XGBClassifier(learning_rate=0.017965145536083, max_depth=11, 
                min_child_weight=1, gamma=0.17426232112256684, 
                colsample_bytree=0.1916286359142644, n_estimators=625, seed=14)

lgbm_mmi = lgb.LGBMClassifier(learning_rate=0.011038126658226765, max_depth=12,
                min_child_samples=5, min_child_weight=0.4318851550452938, min_split_gain=0.025127530387291663,
                n_estimators=471, num_leaves=31) 

svm_mmi = SVC(C=2, kernel='rbf', gamma='scale')

rf_mmi = RandomForestClassifier(max_depth=80, bootstrap=False, max_features = 'sqrt',
                                    n_estimators=400, min_samples_split=3,random_state=0)
lg_mmi = LogisticRegression(penalty='l2', C=1.0, max_iter=1000, solver='lbfgs', random_state=14)

mlp_mmi = MLPClassifier(hidden_layer_sizes=(210,), activation='logistic', solver='adam', random_state=10)

nb_mmi = GaussianNB()  

knn_mmi = KNeighborsClassifier(n_neighbors=7, weights='distance', p=2)  

ada_mmi = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=1),
    n_estimators=100, learning_rate=0.5, random_state=0
)  

dt_mmi = DecisionTreeClassifier(
    max_depth=20, min_samples_split=4, min_samples_leaf=2, random_state=0
)  

et_mmi = ExtraTreesClassifier(
    n_estimators=200, max_depth=15, min_samples_split=4, min_samples_leaf=2,
    max_features='sqrt', bootstrap=True, random_state=0
)  
cb_mmi = CatBoostClassifier(learning_rate=0.25659939142290733, max_depth=4,iterations=90,
                                  boosting_type='Plain')
    
classifiers_mmi = [lgbm_mmi, svm_mmi, lg_mmi, mlp_mmi, knn_mmi, ada_mmi, et_mmi, cb_mmi, nb_mmi, xgb_mmi, rf_mmi]

####

xgb_ncp = XGBClassifier(learning_rate=0.017965145536083, max_depth=11, 
                min_child_weight=1, gamma=0.17426232112256684, 
                colsample_bytree=0.1916286359142644, n_estimators=625, seed=14)

lgbm_ncp = lgb.LGBMClassifier(learning_rate=0.011038126658226765, max_depth=12,
                min_child_samples=5, min_child_weight=0.4318851550452938, min_split_gain=0.025127530387291663,
                n_estimators=471, num_leaves=31) 

svm_ncp = SVC(C=2, kernel='rbf', gamma='scale')

rf_ncp = RandomForestClassifier(max_depth=80, bootstrap=False, max_features = 'sqrt',
                                    n_estimators=400, min_samples_split=3,random_state=0)
lg_ncp = LogisticRegression(penalty='l2', C=1.0, max_iter=1000, solver='lbfgs', random_state=14)

mlp_ncp = MLPClassifier(hidden_layer_sizes=(210,), activation='logistic', solver='adam', random_state=10)

nb_ncp = GaussianNB()  

knn_ncp = KNeighborsClassifier(n_neighbors=7, weights='distance', p=2)  

ada_ncp = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=1),
    n_estimators=100, learning_rate=0.5, random_state=0
)  

dt_ncp = DecisionTreeClassifier(
    max_depth=20, min_samples_split=4, min_samples_leaf=2, random_state=0
)  

et_ncp = ExtraTreesClassifier(
    n_estimators=200, max_depth=15, min_samples_split=4, min_samples_leaf=2,
    max_features='sqrt', bootstrap=True, random_state=0
)  
cb_ncp = CatBoostClassifier(learning_rate=0.25659939142290733, max_depth=4,iterations=90,
                                  boosting_type='Plain')
    
classifiers_ncp = [lgbm_ncp, svm_ncp, lg_ncp, mlp_ncp, knn_ncp, ada_ncp, et_ncp, cb_ncp, nb_ncp, xgb_ncp, rf_ncp]

###
xgb_ps = XGBClassifier(learning_rate=0.017965145536083, max_depth=11, 
                min_child_weight=1, gamma=0.17426232112256684, 
                colsample_bytree=0.1916286359142644, n_estimators=625, seed=14)

lgbm_ps = lgb.LGBMClassifier(learning_rate=0.011038126658226765, max_depth=12,
                min_child_samples=5, min_child_weight=0.4318851550452938, min_split_gain=0.025127530387291663,
                n_estimators=471, num_leaves=31) 

svm_ps = SVC(C=2, kernel='rbf', gamma='scale')

rf_ps = RandomForestClassifier(max_depth=80, bootstrap=False, max_features = 'sqrt',
                                    n_estimators=400, min_samples_split=3,random_state=0)
lg_ps = LogisticRegression(penalty='l2', C=1.0, max_iter=1000, solver='lbfgs', random_state=14)

mlp_ps = MLPClassifier(hidden_layer_sizes=(210,), activation='logistic', solver='adam', random_state=10)

nb_ps = GaussianNB()  

knn_ps = KNeighborsClassifier(n_neighbors=7, weights='distance', p=2)  

ada_ps = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=1),
    n_estimators=100, learning_rate=0.5, random_state=0
)  

dt_ps = DecisionTreeClassifier(
    max_depth=20, min_samples_split=4, min_samples_leaf=2, random_state=0
)  

et_ps = ExtraTreesClassifier(
    n_estimators=200, max_depth=15, min_samples_split=4, min_samples_leaf=2,
    max_features='sqrt', bootstrap=True, random_state=0
)  
cb_ps = CatBoostClassifier(learning_rate=0.25659939142290733, max_depth=4,iterations=90,
                                  boosting_type='Plain')
    
classifiers_ps = [lgbm_ps, svm_ps, lg_ps, mlp_ps, knn_ps, ada_ps, et_ps, cb_ps, nb_ps, xgb_ps, rf_ps]

####

xgb_asdc = XGBClassifier(learning_rate=0.017965145536083, max_depth=11, 
                min_child_weight=1, gamma=0.17426232112256684, 
                colsample_bytree=0.1916286359142644, n_estimators=625, seed=14)

lgbm_asdc = lgb.LGBMClassifier(learning_rate=0.011038126658226765, max_depth=12,
                min_child_samples=5, min_child_weight=0.4318851550452938, min_split_gain=0.025127530387291663,
                n_estimators=471, num_leaves=31) 

svm_asdc = SVC(C=2, kernel='rbf', gamma='scale')

rf_asdc = RandomForestClassifier(max_depth=80, bootstrap=False, max_features = 'sqrt',
                                    n_estimators=400, min_samples_split=3,random_state=0)
lg_asdc = LogisticRegression(penalty='l2', C=1.0, max_iter=1000, solver='lbfgs', random_state=14)

mlp_asdc = MLPClassifier(hidden_layer_sizes=(210,), activation='logistic', solver='adam', random_state=10)

nb_asdc = GaussianNB()  

knn_asdc = KNeighborsClassifier(n_neighbors=7, weights='distance', p=2)  

ada_asdc = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=1),
    n_estimators=100, learning_rate=0.5, random_state=0
)  

dt_asdc = DecisionTreeClassifier(
    max_depth=20, min_samples_split=4, min_samples_leaf=2, random_state=0
)  

et_asdc = ExtraTreesClassifier(
    n_estimators=200, max_depth=15, min_samples_split=4, min_samples_leaf=2,
    max_features='sqrt', bootstrap=True, random_state=0
)  
cb_asdc = CatBoostClassifier(learning_rate=0.25659939142290733, max_depth=4,iterations=90,
                                  boosting_type='Plain')
    
classifiers_asdc = [lgbm_asdc, svm_asdc, lg_asdc, mlp_asdc, knn_asdc, ada_asdc, et_asdc, cb_asdc, nb_asdc, xgb_asdc, rf_asdc]

In [ ]:
def train_and_save_model(classifier, X_train, X_test, y_train, y_test, model_type, fold):
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    
    y_prob = classifier.predict_proba(X_test)[:, 1] if hasattr(classifier, "predict_proba") else None

    #model_path = f'/human_{classifier.__class__.__name__}_{model_type}_fold{fold}.pickle'
    #with open(model_path, 'wb') as model_file:
     #   pickle.dump(classifier, model_file)
    
    return y_pred, y_prob, acc

from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=2, shuffle=True, random_state=4)

datasets = {
    'cksnap': (data_cksnap, labels_cksnap, classifiers_cksnap),
    'Z_curve': (data_z_curve, labels_z_curve, classifiers_Z_curve),
    'kmer': (data_kmer, labels_kmer, classifiers_kmer),
    'mismatch': (data_mismatch, labels_mismatch, classifiers_mismatch),
    'dbe': (data_dbe, labels_dbe, classifiers_dbe),
    'enac': (data_enac, labels_enac, classifiers_enac),
    'nac': (data_nac, labels_nac, classifiers_nac),
    'mmi': (data_mmi, labels_mmi, classifiers_mmi),
    'ncp': (data_ncp, labels_ncp, classifiers_ncp),
    'ps': (data_ps, labels_ps, classifiers_ps),
    'asdc': (data_asdc, labels_asdc, classifiers_asdc),
}

all_y_tests = []

all_preds_list = []
all_probs_list = []
all_preds_dict = {}
all_probs_dict = {}


for fold, (train_index, test_index) in enumerate(kf.split(datasets['cksnap'][0],datasets['cksnap'][1])):
    fold_preds = []  
    fold_probs = []
    
    for model_type, (data, labels, classifiers) in datasets.items():
        X_train, X_test = data[train_index], data[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        
        for classifier in classifiers:
            classifier_name = classifier.__class__.__name__

            y_pred, y_prob, _ = train_and_save_model(classifier, X_train, X_test, y_train, y_test, model_type, fold)
            
            if (model_type, classifier_name) not in all_preds_dict:
                all_preds_dict[(model_type, classifier_name)] = []
                all_probs_dict[(model_type, classifier_name)] = []
            

            all_preds_dict[(model_type, classifier_name)].append(y_pred)
            if y_prob is not None:
                all_probs_dict[(model_type, classifier_name)].append(y_prob)
            
            print(f'{classifier_name} Accuracy {model_type} fold {fold}: {accuracy_score(y_test, y_pred)}')
    all_y_tests.append(y_test)

for (model_type, classifier_name), preds_list in all_preds_dict.items():
    concatenated_preds = np.concatenate(preds_list, axis=0)
    all_preds_list.append(concatenated_preds)

for (model_type, classifier_name), probs_list in all_probs_dict.items():
    if probs_list:
        concatenated_probs = np.concatenate(probs_list, axis=0)
        all_probs_list.append(concatenated_probs)

all_y_testss = np.concatenate(all_y_tests, axis=0)            
all_y_preds = np.column_stack(all_preds_list)
all_y_probs = np.column_stack(all_probs_list) if all_probs_list else None

In [ ]:
from sklearn.linear_model import LogisticRegression
#meta_classifier = LogisticRegression(random_state=0)
meta_classifier = RandomForestClassifier(max_depth=80, bootstrap=True, max_features = 'sqrt',
                                    n_estimators=600, min_samples_split=4,random_state=0)
#meta_classifier = SVC(C=2, kernel='rbf', gamma='scale')

acc_ens_list = []
mcc_ens_list = []
sensitivity_list = []
specificity_list = []
f1_list = []
precision_list = []
recall_list = []
auc_list = []

for fold, (train_index, test_index) in enumerate(kf.split(all_y_preds, all_y_testss)):
    X_train_ens, X_test_ens = all_y_preds[train_index], all_y_preds[test_index]
    y_train_ens, y_test_ens = all_y_testss[train_index], all_y_testss[test_index]

    meta_classifier.fit(X_train_ens, y_train_ens)
    #meta_model_path = f'/human_META_fold{fold}.pickle'
    #with open(meta_model_path, 'wb') as model_file:
     #   pickle.dump(meta_classifier, model_file)
    
    y_pred_ens = meta_classifier.predict(X_test_ens)

    acc_ens = accuracy_score(y_test_ens, y_pred_ens)
    mcc_ens = matthews_corrcoef(y_test_ens, y_pred_ens)

    confusion = confusion_matrix(y_test_ens, y_pred_ens)
    TN, FP, FN, TP = confusion.ravel()
    sensitivity = TP / float(TP + FN)
    specificity = TN / float(TN + FP)
    F1Score = (2 * TP) / float(2 * TP + FP + FN)

    y_pred_prob = meta_classifier.predict_proba(X_test_ens)[:, 1]  
    ROCArea = roc_auc_score(y_test_ens, y_pred_prob)

    acc_ens_list.append(acc_ens)
    mcc_ens_list.append(mcc_ens)
    sensitivity_list.append(sensitivity)
    specificity_list.append(specificity)
    f1_list.append(F1Score)
    auc_list.append(ROCArea)
    
    print(f'Fold {fold + 1} - Acc: {acc_ens}, MCC: {mcc_ens}, Sensitivity: {sensitivity}, Specificity: {specificity}, AUC: {ROCArea}, F1: {F1Score}')

print('Mean Acc Ensemble: ', np.mean(acc_ens_list))
print('Mean Sens Ensemble: ', np.mean(sensitivity_list))
print('Mean Spe Ensemble: ', np.mean(specificity_list))
print('Mean MCC Ensemble: ', np.mean(mcc_ens_list))
print('Mean AUC Ensemble: ', np.mean(auc_list))
print('Mean F1 Ensemble: ', np.mean(f1_list))